In [5]:
from train import *
from helper import criterion
import torch.nn as nn
import torch.nn.functional as F
from rl_model import *
import numpy as np
import torch


In [7]:
client_nums = 25
participant_nums = 5
history_dim = 2
obs_dim = history_dim * (client_nums + 1)
seed = 0
device = 'cuda'
dataset_name = 'CIFAR10'
arch_name = 'CNN'
partition = 'iid'
optimizer = 'SGD'
lr = 0.1
epoch = 1
rl_ddl = 100
batch_size = 32
device = 'cpu'

act_dim = client_nums

print('obs_dim {}, act_dim {}'.format(obs_dim, act_dim))

model = Model(obs_dim, act_dim).to('cuda')
alg = PolicyGradient(model, lr, device=device)
agent = Agent(alg, obs_dim=obs_dim, act_dim=act_dim, participant_nums=participant_nums, client_nums=client_nums, device=device)

obs_dim 52, act_dim 25


In [ ]:
# env = Env(obs_dim, arch_name, client_nums, participant_nums, dataset_name, partition, seed, device, criterion, optimizer="SGD", lr=lr, epoch=epoch, rl_ddl = rl_ddl, batch_size=batch_size)

In [12]:
rewards = torch.ones(batch_size, 1)
rewards[0] = 2
rewards

tensor([[2.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.]])

In [71]:
x = torch.Tensor(batch_size, 1, obs_dim).to('cuda')
actions = torch.ones(batch_size, participant_nums, dtype=torch.long)
rewards = torch.ones(batch_size, 1)
for i in range(batch_size):
    act = np.random.choice(range(client_nums), size=participant_nums, replace=False)
    act = torch.Tensor(act)
    actions[i] = act
y = model(x).cpu()

In [51]:
one_hots = torch.zeros(batch_size, act_dim).scatter(1, torch.LongTensor(actions), torch.ones(batch_size, act_dim))

In [74]:
y = y.squeeze()
y.shape

torch.Size([32, 25])

In [82]:
c = torch.mean(torch.sum(torch.log(y.squeeze()) * one_hots * rewards, dim=1))
c.shape,c


(torch.Size([]), tensor(0., grad_fn=<MeanBackward0>))

In [14]:
type(a) == type(None)

False

In [7]:
obs_list, action_list, reward_list = run_episode(env, agent)
batch_obs = obs_list
batch_action = action_list
batch_reward = calc_reward_to_go(reward_list)

agent.learn(batch_obs, batch_action, batch_reward)

Files already downloaded and verified
Files already downloaded and verified
21:00:36 - root - INFO - Tick 1 Spearman co: -0.02255025502550255
21:00:42 - root - INFO - Tick 2 Spearman co: 0.06567056705670567
21:00:48 - root - INFO - Tick 3 Spearman co: -0.1125112511251125
21:00:53 - root - INFO - Tick 4 Spearman co: -0.024050405040504052
21:00:59 - root - INFO - Tick 5 Spearman co: -0.083012301230123
21:01:05 - root - INFO - Tick 6 Spearman co: 0.027014701470147014
21:01:11 - root - INFO - Tick 7 Spearman co: 0.055733573357335725
21:01:17 - root - INFO - Tick 8 Spearman co: -0.06708670867086708
21:01:23 - root - INFO - Tick 9 Spearman co: 0.009864986498649865
21:01:29 - root - INFO - Tick 10 Spearman co: -0.1242004200420042
21:01:35 - root - INFO - Tick 11 Spearman co: -0.0691029102910291
21:01:41 - root - INFO - Tick 12 Spearman co: -0.016645664566456646
21:01:47 - root - INFO - Tick 13 Spearman co: -0.09044104410441044
21:01:53 - root - INFO - Tick 14 Spearman co: 0.05422142214221421


In [6]:
act_prob =  agent.predict(env.state).detach().squeeze().numpy()

In [14]:
np.count_nonzero(act_prob)

100

In [17]:
np.count_nonzero(ten)

4

In [4]:
obs_list, action_list, reward_list = run_episode(env, agent)

Files already downloaded and verified
Files already downloaded and verified


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument mat1 in method wrapper_CUDA_addmm)

In [5]:
batch_obs = obs_list
batch_action = action_list
batch_reward = calc_reward_to_go(reward_list)

agent.learn(batch_obs, batch_action, batch_reward)

total_reward = evaluate(env, agent, render=False) # render=True 查看渲染效果，需要在本地运行，AIStudio无法显示
spearman_co = spearman(env, agent)
print('Test reward: {}, Spearman co: {}'.format(total_reward, spearman_co))

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Test reward: 0.6296124767150197, Spearman co: -0.09855385538553854


In [ ]:
# alg.model.train()
# cost = 0
# for i in range(len(obs_list)):
#     obs = obs_list[i].detach()
#     action = action_list[i]
#     reward = reward_list[i]

#     act_prob = alg.predict(obs).squeeze()
#     act_dim = alg.model.act_dim
#     one_hot = torch.zeros(act_dim).scatter(0, torch.LongTensor(action), torch.ones(act_dim))
#     cost += torch.sum(-1 * torch.log(act_prob) * one_hot * reward)

In [ ]:
# cost.backward()